In [124]:
#Adapted from code supplements from papers:
#1.`Modewise Operators, the Tensor Restricted Isometry Property, and Low-Rank Tensor Recovery" by Mark A. Iwen, Deanna Needell, Michael Perlmutter, Elizaveta Rebrova
#2.

#Imports
import torch
import torch.nn.functional as F

import tensorly as tl
from tensorly import decomposition
from tensorly import random
from tensorly.decomposition import parafac

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow

from PIL import Image

from scipy.linalg import dft

import timeit

In [125]:
ds=[4] #mode 4 tensors

#can add multiple rs or target_dims to list to run multiple experiments 
ns=[10]  
rs=[2]    #ranks
target_dims=[int(x**2) for x in [40]]   #Target Dimension
meases=["Gaussian"] #Set to either "Gaussian" or "Fourier" 

#number of trials
num_samples=100

#store all parameters in a list of tuples
params=[(n1,d,r1,t,meas) for n1 in ns for d in ds for t in target_dims for r1 in rs for meas in meases]


In [161]:
#Returns low CP rank r tensor of dimension n

def random_low_cp_rank(n,r):
    #torch.manual_seed(0)
    #np.random.seed(0)
    L = []
    for i in range(len(n)):
        C=np.random.normal(0,1,size=(n[i],r))
        L = L + [C]
    
    X = np.zeros(n)
    for i in range(r):
        U_r = np.array(L[0])[:,i]
        for j in range(1, len(n)):
            prod = np.array(L[j])[:,i]
            U_r = np.multiply.outer(U_r,prod)
        X = X + U_r
        
    C=tl.tensor(X) #Changing data frame to tensor
    C.shape
    return C, L


### Some errors in this function, why?
def random_low_cp_rank_1(n,r): ### Issues with printing; also, source of randomness?
    tensor_1 = tl.random.random_cp(shape=n, rank=r)
    return tensor_1

In [162]:
def vectorize(X):
    x=X
    x=x.reshape(-1)
    return x

In [163]:
#Low rank approximation via tensorly Cp decomposition is used for thresholding

def low_rank_approx(tensor,r):    
    #torch.manual_seed(0)
    factors = parafac(tl.tensor(tensor), rank=r)
    answer = tl.cp_to_tensor(factors)
    return answer, factors

In [165]:
tensor, L = random_low_cp_rank((2,5,3),3)

In [171]:
l, a= low_rank_approx(tensor,3)

In [174]:
l-tensor

array([[[ 6.79951649e-06, -1.72809819e-03, -4.26127787e-04],
        [-3.35595196e-05, -5.32636670e-04, -6.41776761e-05],
        [ 5.12836221e-05, -3.37954969e-03, -9.17996088e-04],
        [ 5.75894849e-07,  5.85431224e-04,  1.41713697e-04],
        [ 1.55013106e-05, -2.62579138e-03, -6.46922248e-04]],

       [[ 1.84163180e-05,  6.39714216e-03,  1.61351803e-03],
        [-2.16399367e-05,  1.79682592e-03,  4.51750745e-04],
        [-5.58544190e-04,  1.20493714e-02,  3.33275280e-03],
        [ 1.28337180e-04, -2.01561282e-03, -5.70257676e-04],
        [ 5.23009620e-04,  1.02554725e-02,  2.35693471e-03]]])

In [129]:
#Measurement operators 

#SJLT
def create_SJLT_meas(dim, n):
    #np.random.seed(0)
    return np.sqrt(1/dim)*np.random.normal(0.0, 1.0, [dim, n])


#Gaussian
def create_gaussian_meas(dim, n):
    #np.random.seed(0)
    return np.sqrt(1/dim)*np.random.normal(0.0, 1.0, [dim, n])

#SORS
def create_SORS_meas(dim,n):
    if n<dim:
        raise ValueError("dim is greater than n, matrix needs to be tall and skinny")
    
    n_prod = 1
    for i in n:
        n_prod = n_prod*i   
        
    #np.random.seed(0)
    m=dft(n_prod)/np.sqrt(n_prod)
    vec=np.random.choice([-1,1],n_prod)
    m = np.matmul(m, np.diag(vec))
    m = np.sqrt(n_prod/dim)*m[:int(dim), :]
    return m  

In [130]:
def relative_error(true,guess,first_loss):
    return (np.linalg.norm(true - guess)/first_loss)

In [131]:
np.sqrt(1/6)*np.random.normal(0.0, 1.0, [6, 8])

array([[-0.26150109,  0.43724346,  0.39923038,  0.38642333, -0.37087732,
        -0.42779058, -0.05456498, -0.39196885],
       [-0.09172095, -0.56882438,  0.68621397, -0.35651651, -0.17617069,
        -0.52011357,  0.17653106,  0.48202586],
       [ 1.02176363, -0.47327028,  1.01450405,  0.53059328, -0.07395612,
        -0.18648097,  0.02381816,  0.19947269],
       [ 0.78007322, -0.16755619,  0.29075811,  0.04897233,  0.28792401,
         0.05645328,  0.48702222, -0.07107476],
       [ 0.37899115, -0.32588092, -0.26222688,  0.47376614, -0.43033831,
         0.30894632, -0.05789954, -0.55941463],
       [ 1.20212995, -0.58951792, -0.03546118, -0.19355324, -0.33815828,
         0.36811499, -0.08015499, -0.27671187]])

In [132]:
##Generate Random Tensors
def generate_tensors(n,r,num_samples):

    #torch.manual_seed(0)
    X= []
    for j in range(num_samples):
        X_i= torch.tensor(random_low_cp_rank(n,r), dtype=torch.float64)
        X.append(X_i)
        
    return X

In [133]:
#Vectorized Measurements
def vectorized_measurements(X,dim,num_samples):

    #torch.manual_seed(0)
    n=tuple(X[0].shape)
    n_prod = 1
    for i in n:
        n_prod = n_prod*i
    
    yy = []
    start = timeit.default_timer()
    if meas=="Fourier":
        Afinal=create_kfjl_meas(dim, n_prod)
    elif meas=="Gaussian":
        Afinal=create_gaussian_meas(dim, n_prod)
    else:
        raise ValueError("Set meas to either 'Fourier' or 'Gaussian'")
    
    Afconj = Afinal.conj().T
    for j in range(num_samples):
        y =  np.matmul(Afinal,vectorize(X[j]))
        yy.append(y)
    stop = timeit.default_timer()
    print(Afinal)
    print(y.shape)
    print('Measurement time: ', (stop - start)/num_samples)
    avg_meas_time=(stop - start)/num_samples
    return Afinal,Afconj,yy, avg_meas_time

In [134]:
# CP-TIHT Algorithm

def TIHT(n,r,dim,num_samples=100,mu=.1,N_iter=1000,accuracy=.001):

    #torch.manual_seed(0)

    X = generate_tensors(n,r,num_samples) #generating 100 tensors
    Afinal,Afconj,yy, avg_meas_time=vectorized_measurements(X,dim,num_samples)

    Losses3 = [[1] for _ in range(num_samples)]
    
    good_runs = 0
    total_time = 0
    total_iters = 0

    n=tuple(X[0].shape)

    X0 = torch.randn(n)

    # Run recovery algorithm
    for j in range(num_samples):
        #print(j)
        start = timeit.default_timer()
        X_iter=torch.clone(X0)
        first_loss = np.linalg.norm(X[j] - X_iter)
        i = 0
        while Losses3[j][-1] > accuracy and i < N_iter:
            i += 1 
            Losses3[j].append(relative_error(true=X[j] ,guess=X_iter, first_loss=first_loss))
            measX = np.matmul(Afinal,vectorize(X_iter))
            Z = yy[j] - measX
     
            Z = torch.reshape(torch.tensor(np.matmul(Afconj, Z)), n)
            Y_iter= np.array(X_iter)+mu*np.array(Z)
    
            X_iter=low_rank_approx(Y_iter, r)
        stop = timeit.default_timer()
    
        #plt.plot(range(len(Losses3[j])), Losses3[j])   
        if i < N_iter:
            good_runs += 1
            total_time += stop - start
            total_iters += i
            #print("Converged!")
            #print('Number of iterations: ', i)
    '''if good_runs != 0:
        print('\n')
        print('Percentage of converged runs:', 100*good_runs/num_samples)
        print('Average recovery time: ', total_time/good_runs) 
        print('Average number of iterations: ', total_iters/good_runs) 
    else:
        print("Never converged :(")'''
        
    if good_runs != 0:
        Convergence_percent=100*good_runs/num_samples
        Average_recovery_time= total_time/good_runs
        Average_number_of_iterations= total_iters/good_runs 
   
    else:
        Convergence_percent=0
        Average_recovery_time= np.inf
        Average_number_of_iterations= N_iter
   
    return Convergence_percent, Average_recovery_time, Average_number_of_iterations

In [114]:
X = generate_tensors((2,2,2),2,1) #generating 100 tensors

[array([[[ 0.35449633,  0.11993349],
         [-0.07092101,  0.19813952]],
 
        [[ 0.46661256,  3.24300344],
         [ 0.22353638,  1.03969467]]]),
 array([[[-0.8571351 ,  2.45645279],
         [ 0.47169531, -1.28524858]],
 
        [[-0.43872991,  0.9125632 ],
         [ 0.62006192, -1.48858708]]]),
 array([[[-0.71548326,  0.82503068],
         [-0.05711954,  0.06038757]],
 
        [[-3.64176973,  4.83045518],
         [-0.30838667,  0.68348106]]]),
 array([[[-0.5351592 ,  0.51296666],
         [ 0.1760892 , -0.30951059]],
 
        [[ 0.39345461, -0.91989999],
         [-1.54001826, -2.20014139]]])]

In [159]:
Convergence_percent, Average_recovery_time, Average_number_of_iterations = TIHT((2,2,2),2,8,num_samples=3,mu=.1,N_iter=1000,accuracy=.001)

[[ 0.06005319  1.04013912 -0.83005407 -0.46835473 -0.17545923 -0.06272037
  -0.00579962  0.9029238 ]
 [ 0.62436748 -0.0349792   0.11811999  0.75464745 -0.17349836  0.04115035
  -0.08649229 -0.45901935]
 [-0.63415078 -0.2421735   0.2879426   0.11248248  0.67503637  0.28588326
   0.27587621 -0.03133723]
 [-0.09152512  0.02457839  0.1296615  -0.13301549  0.07336067 -0.1207284
   0.04894593 -0.27776161]
 [ 0.1830972   0.48190246 -0.22924035  0.28540162 -0.11699345  0.09660505
   0.21049334  0.17502218]
 [ 0.16708231 -0.28756904  0.1425399  -0.29922317 -0.44174516 -0.27543545
  -0.1061101   0.26893757]
 [-0.92051604  0.08175615  0.02198367  0.46577936 -0.2086078   0.39269736
   0.37061337 -0.10227804]
 [ 0.07820479 -0.30262484  0.00408653 -0.33010135  0.68808749 -0.75166663
   0.10426023  0.44069024]]
torch.Size([8])
Measurement time:  0.0004159306666527603


/var/folders/1m/d1lrmr2n7gvdmlys8cnm3nt40000gn/T/ipykernel_1852/4146494319.py:33: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  Z = torch.reshape(torch.tensor(np.matmul(Afconj, Z)), n)


In [160]:
Convergence_percent

0

In [48]:
meas = "Gaussian"
Afinal,Afconj,yy, avg_meas_time=vectorized_measurements(X,6,4)

[[ 0.2613326  -0.66011954 -0.0099311  -0.30129986  0.11427874 -0.04006973
   0.37157898  0.12950379]
 [ 0.32101705 -0.1904148  -0.38556857 -0.16740209 -0.00694855  0.15478805
   0.92235902 -0.01725141]
 [-0.39026291 -0.14124647 -0.18926226  0.19656399 -0.62902775  0.0258266
   0.06389353  0.09478751]
 [-0.24385326 -0.09713114 -0.58137043 -0.201397   -0.22162227  0.16985172
  -0.4720095   0.31892279]
 [ 0.61012076 -0.84506785  0.1740194   0.27634655 -0.26023258 -0.16218554
  -0.05424827 -0.12157262]
 [-0.12615402 -0.68422569  0.47043739  0.44075244 -0.33205457 -0.59866522
   0.21272385 -0.23506445]]
(6,)
Measurement time:  0.00023126049995880749


In [151]:
n=tuple(X[0].shape)

X0 = torch.randn(n)
Losses3 = [[1] for _ in range(num_samples)]
j = 0
mu = 0.1
r = 2

In [156]:
        start = timeit.default_timer()
        X_iter=np.array(torch.clone(X0))
        first_loss = np.linalg.norm(X[0] - X_iter)
        i = 0
        while Losses3[0][-1] > 0.01 and i < 10000:
            i += 1 
            Losses3[j].append(relative_error(true=X[0] ,guess=X_iter, first_loss=first_loss))
            measX = np.matmul(Afinal,vectorize(X_iter))
            Z = yy[0] - measX
     
            Z = torch.reshape(torch.tensor(np.matmul(Afconj, Z)), n)
            Y_iter= np.array(X_iter)+mu*np.array(Z)
    
            X_iter=low_rank_approx(Y_iter, r)
        stop = timeit.default_timer()
    
        #plt.plot(range(len(Losses3[j])), Losses3[j])   
        if i < 1000:
            good_runs += 1
            total_time += stop - start
            total_iters += i
            print("Converged!")
            print('Number of iterations: ', i)


In [157]:
Losses3[0]

[1,
 1.0,
 1.0405261818444624,
 1.1013831828733711,
 1.1662751134918015,
 1.228223119519232,
 1.2846896050781762,
 1.3350971360611248,
 1.3796768025718833,
 1.418948278127542,
 1.4534754329684618,
 1.4837920842405852,
 1.5104368476042083,
 1.533938501294389,
 1.5547551420911576,
 1.5732496148510464,
 1.5897051740619157,
 1.604352207972432,
 1.6173893066616551,
 1.6289945213218204,
 1.6393286762921406,
 1.6485353104256781,
 1.6567409487135836,
 1.6640565318922769,
 1.6705793294536255,
 1.6763947831984178,
 1.6815781148613247,
 1.686195695658868,
 1.6903062102923372,
 1.6939616486823739,
 1.6972081539914405,
 1.700086750559891,
 1.702633971077896,
 1.7048823987638657,
 1.706861137467471,
 1.708596220348785,
 1.7101109659836697,
 1.711426289304007,
 1.712560973616241,
 1.7135319089897023,
 1.7143543015202913,
 1.715041857320643,
 1.7156069445387139,
 1.7160607362432527,
 1.7164133366215006,
 1.7166738925998497,
 1.716850692712447,
 1.7169512547981618,
 1.7169824038965174,
 1.7169503415329

In [15]:
np.zeros([4,2,3,5])

array([[[[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]]],


       [[[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]]],


       [[[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]]],


       [[[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]]]])

In [ ]:
#### Attempt TIHT for Cp ranks

## Rank